In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader,sampler,Dataset
import torchvision.datasets as dset
import torchvision.transforms as T
import timeit
from PIL import Image
import os
import numpy as np
import scipy.io

In [2]:
dtype = torch.FloatTensor # the CPU datatype
# Constant to control how frequently we print train loss
print_every = 100
# This is a little utility that we'll use to reset the model
# if we want to re-initialize all our parameters
def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

In [3]:
torch.cuda.is_available()

True

In [7]:
class Flatten3d(nn.Module):
    def forward(self, x):
        ###############6th TODO (5 points)###################
        N, P, C, H, W = x.size() 
        return x.view(N, -1) 

In [8]:
fixed_model_3d_frame = nn.Sequential( # You fill this in!
    ###############7th TODO (20 points)#########################
    nn.Conv3d(3, 64, kernel_size=3, padding=1),
    nn.BatchNorm3d(64),
    nn.ReLU(),
    nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2)),
    nn.Conv3d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm3d(128),
    nn.ReLU(),
    nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)),
    Flatten3d(),
    nn.Linear(32768, 2048),   
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(2048, 10),
)

fixed_model_3d = fixed_model_3d_frame.type(dtype)
x = torch.randn(32,3, 3, 64, 64).type(dtype)
x_var = Variable(x).type(dtype) # Construct a PyTorch Variable out of your input data
ans = fixed_model_3d(x_var) 
np.array_equal(np.array(ans.size()), np.array([32, 10]))

True

In [9]:
import copy
gpu_dtype = torch.cuda.FloatTensor

fixed_model_gpu = copy.deepcopy(fixed_model_3d)#.type(gpu_dtype)
fixed_model_gpu.cuda()
x_gpu = torch.randn(32,3,3,64,64).cuda()#.type(gpu_dtype)
x_var_gpu = Variable(x_gpu).type(gpu_dtype) # Construct a PyTorch Variable out of your input data
ans = fixed_model_gpu(x_var_gpu)        # Feed it through the model! 

# Check to make sure what comes out of your model
# is the right dimensionality... this should be True
# if you've done everything correctly
np.array_equal(np.array(ans.size()), np.array([32, 10]))

True

In [10]:
label_mat=scipy.io.loadmat('./data/hw6_data.mat')
label_train=label_mat['trLb']
print(len(label_train))
label_val=label_mat['valLb']
print(len(label_val))

7770
2230


In [11]:
clip_dataloader = DataLoader(clip_dataset, batch_size=4,
                        shuffle=True, num_workers=4)


for i,sample in enumerate(clip_dataloader):
    print(i,sample['clip'].shape,sample['folder'],sample['Label'])
    if i>20: 
        break

NameError: name 'clip_dataset' is not defined

In [68]:
clip_dataset_train=ActionClipDataset(root_dir='./data/trainClips/',labels=label_train,transform=T.ToTensor())

clip_dataloader_train = DataLoader(clip_dataset_train, batch_size=16,
                        shuffle=True, num_workers=4)
clip_dataset_val=ActionClipDataset(root_dir='./data/valClips/',labels=label_val,transform=T.ToTensor())

clip_dataloader_val = DataLoader(clip_dataset_val, batch_size=16,
                        shuffle=True, num_workers=4)
clip_dataset_test=ActionClipDataset(root_dir='./data/testClips/',labels=[],transform=T.ToTensor())

clip_dataloader_test = DataLoader(clip_dataset_test, batch_size=16,
                        shuffle=False, num_workers=4)

In [69]:
dtype = torch.cuda.FloatTensor # the CPU datatype
# Constant to control how frequently we print train loss
print_every = 100
# This is a little utility that we'll use to reset the model
# if we want to re-initialize all our parameters
def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

In [124]:
def train_3d(model, loss_fn, optimizer,dataloader,num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, sample in enumerate(dataloader):
            x_var = Variable(sample['clip'].type(dtype))
            y_var = Variable(sample['Label'].type(dtype).long())

            scores = model(x_var)
            
            loss = loss_fn(scores, y_var)
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        check_accuracy_3d(model, clip_dataloader_train) 
        check_accuracy_3d(model, clip_dataloader_val) 
        torch.save(model.state_dict(), os.path.join('./', 'epoch-{}.pth'.format(epoch)))
        

def check_accuracy_3d(model, loader):
    '''
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')  
    '''
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for t, sample in enumerate(loader):
        x_var = Variable(sample['clip'].type(dtype))
        y_var = sample['Label'].type(dtype)
        y_var=y_var.cpu()
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        #print(preds)
        #print(y_var)
        num_correct += (preds.numpy() == y_var.numpy()).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

transform_TJ = T.Compose([
    #transforms.ToPILImage(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [72]:
class ActionClipDataset(Dataset):
    """Action Landmarks dataset."""

    def __init__(self,  root_dir,labels=[], transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        
        self.root_dir = root_dir
        self.transform = transform
        self.length=len(os.listdir(self.root_dir))
        self.labels=labels

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        
        folder=idx+1
        folder=format(folder,'05d')
        clip=[]
        if len(self.labels)!=0:
            Label=self.labels[idx][0]-1
        for i in range(3):
            imidx=i+1
            imgname=str(imidx)+'.jpg'
            img_path = os.path.join(self.root_dir,
                                    folder,imgname)
            image = Image.open(img_path)
            image=np.array(image)
            clip.append(image)
        if self.transform:
            clip=np.asarray(clip)
            clip=np.transpose(clip, (0,3,1,2))
            clip = torch.from_numpy(np.asarray(clip))
        if len(self.labels)!=0:
            sample={'clip':clip,'Label':Label,'folder':folder}
        else:
            sample={'clip':clip,'folder':folder}
        return sample

clip_dataset=ActionClipDataset(root_dir='./data/trainClips/',\
                               labels=label_train,transform=T.ToTensor())#/home/tqvinh/Study/CSE512/cse512-s18/hw2data/trainClips/
for i in range(10):
    sample=clip_dataset[i]
    print(sample['clip'].shape)
    print(sample['Label'])
    print(sample['folder'])

torch.Size([3, 3, 64, 64])
0.0
00001
torch.Size([3, 3, 64, 64])
0.0
00002
torch.Size([3, 3, 64, 64])
0.0
00003
torch.Size([3, 3, 64, 64])
0.0
00004
torch.Size([3, 3, 64, 64])
0.0
00005
torch.Size([3, 3, 64, 64])
0.0
00006
torch.Size([3, 3, 64, 64])
0.0
00007
torch.Size([3, 3, 64, 64])
0.0
00008
torch.Size([3, 3, 64, 64])
0.0
00009
torch.Size([3, 3, 64, 64])
0.0
00010


In [73]:
clip_dataloader = DataLoader(clip_dataset, batch_size=4,
                        shuffle=True, num_workers=4)


for i,sample in enumerate(clip_dataloader):
    print(i,sample['clip'].shape,sample['folder'],sample['Label'])
    if i>20: 
        break

(0, torch.Size([4, 3, 3, 64, 64]), ['01670', '01838', '05799', '02410'], tensor([1., 2., 7., 2.], dtype=torch.float64))
(1, torch.Size([4, 3, 3, 64, 64]), ['03730', '01299', '05095', '00772'], tensor([4., 1., 6., 0.], dtype=torch.float64))
(2, torch.Size([4, 3, 3, 64, 64]), ['00960', '03267', '00615', '07076'], tensor([1., 3., 0., 8.], dtype=torch.float64))
(3, torch.Size([4, 3, 3, 64, 64]), ['02341', '01900', '04565', '06235'], tensor([2., 2., 5., 7.], dtype=torch.float64))
(4, torch.Size([4, 3, 3, 64, 64]), ['04065', '04808', '00025', '06827'], tensor([4., 5., 0., 8.], dtype=torch.float64))
(5, torch.Size([4, 3, 3, 64, 64]), ['03175', '05810', '07177', '06202'], tensor([3., 7., 9., 7.], dtype=torch.float64))
(6, torch.Size([4, 3, 3, 64, 64]), ['01894', '00710', '05611', '05043'], tensor([2., 0., 6., 6.], dtype=torch.float64))
(7, torch.Size([4, 3, 3, 64, 64]), ['05726', '04530', '05464', '01105'], tensor([7., 5., 6., 1.], dtype=torch.float64))
(8, torch.Size([4, 3, 3, 64, 64]), ['031

In [74]:
clip_dataset_train=ActionClipDataset(root_dir='./data/trainClips/',labels=label_train,transform=T.ToTensor())

clip_dataloader_train = DataLoader(clip_dataset_train, batch_size=16,
                        shuffle=True, num_workers=4)
clip_dataset_val=ActionClipDataset(root_dir='./data/valClips/',labels=label_val,transform=T.ToTensor())

clip_dataloader_val = DataLoader(clip_dataset_val, batch_size=16,
                        shuffle=True, num_workers=4)
clip_dataset_test=ActionClipDataset(root_dir='./data/testClips/',labels=[],transform=T.ToTensor())

clip_dataloader_test = DataLoader(clip_dataset_test, batch_size=16,
                        shuffle=False, num_workers=4)

In [75]:
class Flatten3d(nn.Module):
    def forward(self, x):
        ###############6th TODO (5 points)###################
        N, P, C, H, W = x.size() 
        return x.view(N, -1) 

In [152]:
fixed_model_3d_frame = nn.Sequential( # You fill this in!
    ###############7th TODO (20 points)#########################
    nn.Conv3d(3, 8, kernel_size=(1,3,3), stride=1),
    nn.BatchNorm3d(8),
    nn.ReLU(inplace=True),
    nn.MaxPool3d(kernel_size=(3, 2, 2), stride=(1,2,2), padding=1),
    
    # conv 2
    nn.Conv3d(8, 16, kernel_size=(1,3,3), stride=1),
    nn.BatchNorm3d(16),
    nn.ReLU(inplace=True),
    nn.MaxPool3d(kernel_size=(3, 2, 2), stride=(1,2,2), padding=1),
     # conv 3
    nn.Conv3d(16, 32, kernel_size=(1,3,3), stride=1),
    nn.BatchNorm3d(32),
    nn.ReLU(inplace=True),
    nn.MaxPool3d(kernel_size=(3, 2, 2), stride=(1,2,2), padding=1),       
    # conv 4-1
    nn.Conv3d(32, 64, kernel_size=(1,3,3), stride=1),
    nn.BatchNorm3d(64),
    nn.ReLU(inplace=True),
    nn.MaxPool3d(kernel_size=(3, 2, 2), stride=(1,2,2), padding=1),  
    nn.Dropout(p=0.5),
    # conv 4-2
    nn.Conv3d(64, 64, kernel_size=(1,3,3), stride=1),
    nn.BatchNorm3d(64),
    nn.ReLU(inplace=True),
    nn.MaxPool3d(kernel_size=(3, 2, 2), stride=(1,2,2), padding=1),
    Flatten3d(),
    nn.Linear(768,10),
    #nn.ReLU(inplace=True),
    #nn.Dropout(p=0.5),
    #nn.Linear(64,10)
)

fixed_model_3d = fixed_model_3d_frame.type(dtype)
x = torch.randn(32,3, 3, 64, 64).type(dtype)
x_var = Variable(x).type(dtype) # Construct a PyTorch Variable out of your input data
ans = fixed_model_3d(x_var) 
np.array_equal(np.array(ans.size()), np.array([32, 10]))


True

In [153]:
loss_fn = nn.CrossEntropyLoss().cuda()
optimizer = optim.RMSprop(fixed_model_gpu.parameters(), lr=1e-4)

In [154]:
torch.cuda.random.manual_seed(12345)

fixed_model_gpu.apply(reset) 
fixed_model_gpu.train() 
train_3d(fixed_model_gpu, loss_fn, optimizer,clip_dataloader_train, num_epochs=5) 
check_accuracy_3d(fixed_model_gpu, clip_dataloader_val)# check accuracy on the training set


Starting epoch 1 / 5


/home/xiaolinghu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


t = 100, loss = 0.9059
t = 200, loss = 1.2550
t = 300, loss = 0.2361
t = 400, loss = 0.1434
Got 7166 / 7770 correct (92.23)
Got 1448 / 2230 correct (64.93)
Starting epoch 2 / 5
t = 100, loss = 0.1000
t = 200, loss = 0.3157
t = 300, loss = 0.0326
t = 400, loss = 0.0257
Got 7679 / 7770 correct (98.83)
Got 1432 / 2230 correct (64.22)
Starting epoch 3 / 5
t = 100, loss = 0.0439
t = 200, loss = 0.0617
t = 300, loss = 0.2620
t = 400, loss = 0.0036
Got 7741 / 7770 correct (99.63)
Got 1435 / 2230 correct (64.35)
Starting epoch 4 / 5
t = 100, loss = 0.1341
t = 200, loss = 0.0199
t = 300, loss = 0.1130
t = 400, loss = 0.0079
Got 7763 / 7770 correct (99.91)
Got 1496 / 2230 correct (67.09)
Starting epoch 5 / 5
t = 100, loss = 0.3267
t = 200, loss = 0.0014
t = 300, loss = 0.0087
t = 400, loss = 0.0004
Got 7768 / 7770 correct (99.97)
Got 1535 / 2230 correct (68.83)
Got 1535 / 2230 correct (68.83)


In [155]:
def predict_on_test_3d(model, loader):
    '''
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')  
    '''
    num_correct = 0
    num_samples = 0
    #model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results_3d.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for t, sample in enumerate(loader):
        x_var = Variable(sample['clip'].type(dtype))
        scores = model(x_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i])+'\n')
            count+=1
    results.close()
    return count
    
count=predict_on_test_3d(fixed_model_gpu, clip_dataloader_test)
print(count)

3270


In [3]:
import torchvision.models as models

In [6]:
torch.load('./resnet-18-kinetics.pth')

IOError: [Errno 2] No such file or directory: '/Users/tingjin_jane/Downloads/resnet-18-kinetics.pth'